# Survey non-uniformity check pipeline:

This is the draft RAIL pipeline for checking, given choice of photo-z pipeline, the effect of survey non-uniformity.

for this also c.f. `golden spike rail_pipelines` under the DESC directory making a pipeline class

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import healpy as hp
import pickle
import pandas as pd
from collections import OrderedDict
import yaml

import matplotlib
cmap = matplotlib.cm.get_cmap('plasma')

In [ ]:
# RAIL modules:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

DS = RailStage.data_store
DS.__class__.allow_overwrite = True

#import pzflow
#from pzflow import Flow
from rail.creation.engines.flowEngine import FlowCreator

from rail.creation.degradation import observing_condition_degrader
from rail.creation.degradation.observing_condition_degrader import ObsCondition


import tables_io

from rail.estimation.algos.flexzboost import Inform_FZBoost, FZBoost
from rail.estimation.algos.bpz_lite import BPZ_lite

In [ ]:
# for dust (de-)reddening:
import dustmaps
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord
from dustmaps.config import config
config['data_dir'] = '/global/cfs/cdirs/lsst/groups/PZ/PhotoZDC2/run2.2i_dr6_test/TESTDUST/mapdata' 
# above path may be / have been updated...

User defined functions:

In [ ]:
# step 0.5: append semi-major/minor axis of the galaxy from size and ellipticity

def get_semi_major_minor(catalog, scale=1):

    q = (1 - catalog['ellipticity'])/(1 + catalog['ellipticity'])
    ai = catalog['size']
    bi = ai*q

    ai = ai.to_numpy()*scale
    bi = bi.to_numpy()*scale
    
    d = {"major": ai, "minor": bi}
    major_minor_axis = pd.DataFrame(data=d)
    
    # append to the input data
    catalog = pd.concat([catalog, major_minor_axis], axis=1)
    
    return catalog

In [ ]:
def deredden_galaxy(data, bands, nside=128):
    
    # set the A_lamba/E(B-V) values for the six LSST filters 
    band_a_ebv = np.array([4.81,3.64,2.70,2.06,1.58,1.31])
    
    # turn assigned pixels to ra, dec:
    assigned_pixels = data["pixels"]
    ra, dec = hp.pix2ang(nside, assigned_pixels, lonlat=True)
    coords = SkyCoord(ra, dec, unit = 'deg', frame='fk5')
    # compute EBV
    sfd = SFDQuery()
    ebvvec = sfd(coords)

    mag_dered = pd.DataFrame()
    for ii, b in enumerate(bands):
        mag_dered[b] = data[b]-ebvvec*band_a_ebv[ii]
    
    return mag_dered

In [ ]:
def convert_catalog_to_test_data(data, DS, bands, nside=128, dered=False):

    data2 = OrderedDict()
    
    if dered == True:
        mag_dered = deredden_galaxy(data, bands, nside=nside)
    
    for bb in bands:
        data2['%s_err'%bb] = data['%s_err'%bb].to_numpy()
        if dered == False:
            data2[bb] = data[bb].to_numpy()
        elif dered == True:
            data2[bb] = mag_dered[bb].to_numpy()
        
    data2['redshift'] = data['redshift'].to_numpy()

    xtest_data = tables_io.convert(data2, tables_io.types.NUMPY_DICT)
    test_data = DS.add_data("test_data", xtest_data, TableHandle)
    
    return test_data

In [ ]:
# this is the simplist binning method using the pz point estimate directly:

def assign_lens_bins(catalog, nYrObs=1):
    """
    nYrObs: lens binning requirement to adopt.
    """
    pz = catalog['pz_point'].to_numpy()
    if nYrObs==1:
        # adopt 1 year criteria: 5 bins between 0.2<z<1.2
        bin_edges = np.linspace(0.2,1.2,6)
        bin_index = np.digitize(pz, bin_edges) 
    if nYrObs==10:
        # adopt 10 year criteria: 10 bins between 0.2<z<1.2
        bin_edges = np.linspace(0.2,1.2,11)
        bin_index = np.digitize(pz, bin_edges) 

    # append the tomographic binning
    tomo = pd.DataFrame(data={"tomo": bin_index})
    catalog_tomo = pd.concat([catalog, tomo], axis=1)
    # trim the catalogue for objects not in the bin:
    sel = np.where((bin_index>=1)&(bin_index<len(bin_edges)))[0]
    catalog_tomo = catalog_tomo[sel]
    
    return catalog_tomo


def assign_source_bins(catalog):
    # requirement is 5 bins with equal number of objects:
    # here we just use simple quantile from the estimated photo-z
    pz = catalog['pz_point'].to_numpy()
    nbins=5
    sortind = np.sortarg(pz)
    N_perbin = int(len(pz)/nbins)
    bin_index = np.zeros(len(pz))
    for ii in range(nbins):
        useind = sortind[ii*N_perbin:(ii+1)*N_perbin]
        bin_index[useind] = ii+1
    
    # append the tomographic binning
    tomo = pd.DataFrame(data={"tomo": bin_index})
    catalog_tomo = pd.concat([catalog, tomo], axis=1)
    return catalog_tomo



In [ ]:
def split_sys_map_quantiles(mapin, mask, nquantiles=10):
    """
    mapin: systematic map input
    mask: mask for the systematic map
    nquantiles: number of quantiles to split into, default is 10
    """
    
    pix = np.arange(len(mapin))
    usemap = mapin[mask.astype(bool)]
    usepix = pix[mask.astype(bool)]

    sortind = np.argsort(usemap)
    Npix_perq = int(len(usemap)/nquantiles)
    
    quantile=[]
    meanv = np.zeros(len(nquantiles))
    
    for ii in range(nquantiles):
        useind = sortind[ii*Npix_perq:(ii+1)*Npix_perq]
        pix = usepix[useind]
        #temp = np.zeros(len(mask))
        #temp[pix]=1
        quantile.append(pix)
        # compute the mean value of the quantile:
        meanv[ii] = np.mean(mapin[pix])
    return quantile, meanv


def compute_nzstats(usecat, z_col, zgrid=np.linspace(0,1,31), nbootstrap=None):
    """
    Computes some summary statistics of the n_z for input catalog, specifically:
    - nz: given zgrid, output will be [zcentre, Nz]
    - meanz: given zgrid, weighted mean redshift with bootstrap error [meanz, meanz_err]
    - sigmaz: given zgrid, weighted first moment of z (sigmaz) with bootstrap error [sigmaz, sigmaz_err]
    """
    
    Nz = np.histogram(usecat[z_col], bins=zgrid)
    zcentre = (zgrid[1:] + zgrid[:-1])*0.5
    nz = np.c_[zcentre, Nz]
    
    meanz = np.sum(Nz*zcentre)/np.sum(Nz)
    
    sigmaz = np.sqrt(np.sum(Nz*(zcentre-meanz)**2)/np.sum(Nz))
    
    # compute the bootstrap error:
    sampholder_meanz = np.zeros(nbootstrap)
    sampholder_sigmaz = np.zeros(nbootstrap)
    
    for kk in range(nbootstrap):
        samp = np.random.choice(usecat[z_col], 
                        size=len(usecat[z_col]),
                        replace=True)
        # repeat the operation 
        cc = np.histogram(samp,bins=zgrid)
        sampholder_meanz[kk] = np.sum(cc[0] * zcentre)/np.sum(cc[0])
        sampholder_sigmaz[kk] = np.sqrt(np.sum(cc[0]*(zcentre-meanz)**2)/np.sum(cc[0]))
        
    meanz_err = np.std(sampholder_meanz)
    sigmaz_err = np.std(sampholder_sigmaz)
    
    meanz = np.array([meanz, meanz_err])
    sigmaz = np.array([sigmaz, sigmaz_err])
    
    return nz, meanz, sigmaz

def write_evaluation_results(outroot, meanv, nzstat_summary_split, nzstat_summary_tot):
    
    ntomo = list(nzstat_summary_split.keys())
    
    out = {}
    
    out["nquantile"] = len(meanv)
    out["mean_systematic"] = meanv
    
    for jj in ntomo:
        out["tomo-%d"%(jj+1)]={
            "nz": [],
            "meanz": np.array((len(meanv),2)),
            "sigmaz": np.array((len(meanv),2)),
        }
        for kk in range(len(meanv)):
            nz, meanz, sigmaz = nzstat_summary_split["tomo-%d"%(jj+1)][kk]
            out["tomo-%d"%(jj+1)]["nz"].append(nz) 
            out["tomo-%d"%(jj+1)]["meanz"][kk,:] = meanz
            out["tomo-%d"%(jj+1)]["sigmaz"][kk,:] = sigmaz
            
        # add unbinned stats:
        nz, meanz, sigmaz = nzstat_summary_tot["tomo-%d"%(jj+1)]
        out["tomo-%d"%(jj+1)]["nztot"]=nz 
        out["tomo-%d"%(jj+1)]["meanztot"] = meanz
        out["tomo-%d"%(jj+1)]["sigmaztot"] = sigmaz
            
    # save to yaml file
    file=open(outroot,"w")
    yaml.dump(out,file)
    file.close()

## Pre-defined variables:

In [ ]:
# pretrained flows stored in here
photFlow_dir = 'main_galaxy_flow/flow.pzflow.pkl'
shapeFlow_dir = 'conditional_galaxy_flow/flow.pzflow.pkl'

In [ ]:
# first define a set of input map directories:

base_path = "/pscratch/sd/q/qhang/rubin_baseline_v2/MAF-1year/"

# nside of these maps:
nside=128

# seeing maps:
seeing_u = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_u_and_nightlt365_HEAL.fits"
seeing_g = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_g_and_nightlt365_HEAL.fits"
seeing_r = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_r_and_nightlt365_HEAL.fits"
seeing_i = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_i_and_nightlt365_HEAL.fits"
seeing_z = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_z_and_nightlt365_HEAL.fits"
seeing_y = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_y_and_nightlt365_HEAL.fits"

# coadd depth maps:
coaddm5_u = base_path + "baseline_v2_0_10yrs_CoaddM5_u_and_nightlt365_HEAL.fits"
coaddm5_g = base_path + "baseline_v2_0_10yrs_CoaddM5_g_and_nightlt365_HEAL.fits"
coaddm5_r = base_path + "baseline_v2_0_10yrs_CoaddM5_r_and_nightlt365_HEAL.fits"
coaddm5_i = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
coaddm5_z = base_path + "baseline_v2_0_10yrs_CoaddM5_z_and_nightlt365_HEAL.fits"
coaddm5_y = base_path + "baseline_v2_0_10yrs_CoaddM5_y_and_nightlt365_HEAL.fits"

# here we will set the observing year and number of visits per year to 1, because we are supplying coadd depth

# mask:
maskdir = base_path + "../wfd_footprint_nvisitcut_500_nside_128.fits"

# weight: for now we supply uniform weight

# choose the systematic map to examine, here we choose the combined depth:
sys_to_check = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
sys = "CoaddM5"

# directory to save all the data:
savedir = "/pscratch/sd/q/qhang/PZflow-samples/DC2-test/"

In [ ]:
# calibration of the error model 
# magerrscale currently not implemented, 
# probably need to modify the pipeline or save a new set of sys maps

lsstError_calibration={
    'semi_major_minor_scale':1/2.5,
    'magerrscale':{
        'u': 0.73,
        'g': 1.20,
        'r': 0.98, 
        'i': 1.10,
        'z': 1.10,
        'y': 1.15,
    },
}

## Step 1: RAIL.creation

### 1.1 Creation:
In this example, we load pre-trained flow:

In [ ]:
# load the pzflow
# below need testing to see if it works (original code works on pzflow)
n_samples = 100000 #1e5 galaxies
photFlow = FlowCreator.make_stage(name='photFlow', model=photFlow_dir, n_samples=n_samples)
shapeFlow = FlowCreator.make_stage(name='shapeFlow', model=shapeFlow_dir, n_samples=n_samples)

In [ ]:
# check if the model is read in correctly:
photFlow.get_data('model')

In [ ]:
# define number of galaxies to generate:
# still need to check if this line works...(original code works on pzflow)
photoCat = photFlow.sample(n_samples, seed=0)
fullCat = shapeFlow.sample(conditions=photoCat, seed=0)

print(fullCat())
print("Data was written to ", fullCat.path)
# the fullCat should now be stored in the data Handle

### 1.2 Degradation:
Use the degradation to specify a set of systematic maps, and generate observed magnitudes and magnitude errors:
- Generate magnitude error according to survey condition maps supplied, generate degraded mag + magerr
    - Note: galactic extinction to the cosmoDC2 magnitudes are not applied yet, probably need pipeline expansion to do it
    - Note: currently `magerrscale` cannot be applied
    - Note: can also specify the detection limit via `sigLim` (set to 3 sigma here);
- Select data with i-band detection and flag non-detection in other bands as `np.nan`, apply further cuts such as the gold cut $i<23.5$.

In [ ]:
# Another option for including the magerrscale correction: 
# Adjust the m5 maps by:
# m5 = m5 - 2.5*np.log10(lsstError_calibration['magerrscale'][band])
# but currently we read maps via input file name
# so maybe we should modify obs_cond to directly take arrays (maps)

In [ ]:
# Create the degrader
# list of arguement here
# Note: number of years and visits set to 1 because coadd depth are supplied for m5
y1_degrader = ObsCondition.make_stage(
    map_dict={
        "theta": {
            "u": seeing_u,
            "g": seeing_g,
            "r": seeing_r,
            "i": seeing_i,
            "z": seeing_z,
            "y": seeing_y,
        },
        "m5": {
            "u": coaddm5_u,
            "g": coaddm5_g,
            "r": coaddm5_r,
            "i": coaddm5_i,
            "z": coaddm5_z,
            "y": coaddm5_y,
        },
        "nYrObs": 1.,
        "nVisYr": {
            "u": 1.,
            "g": 1.,
            "r": 1.,
            "i": 1.,
            "z": 1.,
            "y": 1., 
        },
        "sigLim": 3,
        "ndFlag": np.nan,
        "extendedType": "auto",
        "majorCol": "major",
        "minorCol": "minor",
        "decorrelate": True,
        "highSNR": False,
    },
    nside=nside,
    mask = maskdir,
    weight = "",
)

# Compute the semi major and minor axes
fullCat = get_semi_major_minor(catalog, scale=lsstError_calibration["semi_major_minor_scale"])

# degraded data below
data_degraded = y1_degrader(fullCat)

# calibrated LSST error model:
# partly done in the semi-major axis part
# here apply the overall factor:
# lsstError_calibration['magerrscale'] 
# need to think about this step further, perhaps use a set up maps corrected for it.

# selection based on non-observation in i-band:
nan_index = np.isnan(data_degraded.data['i'])

# gold cut:
goldsel = data_degraded.data['i']<25.3

# apply selections:
data_degraded_gold = data_degraded.data[goldsel*(~nan_index)]
# reset the index
data_degraded_gold = data_degraded_gold.data.reset_index()

# print number of objects:
print("Number of objects selected in catalogue: ", len(data_degraded_gold))

## Step 2: RAIL.estimation

Below is an example of using `BPZ_lite` to estimate redshifts for the above sample
- First de-redden the magnitudes
- Point estimate of the BPZ redshifts by extracting redshift mode

In [ ]:
# BPZ_lite version:

band_names = ['u','g','r','i','z','y']
band_err_names = ['u_err','g_err','r_err','i_err','z_err','y_err']
prior_band='i'

output = savedir + "BPZ_lite_photoz.hdf5"

estimate_bpz = BPZ_lite.make_stage(name='estimate_bpz', hdf5_groupname='', 
                                   #columns_file=inroot+'test_bpz.columns',
                                   #prior_file='CWW_HDFN_prior.pkl',
                                   nondetect_val=np.nan, #spectra_file='SED/CWWSB4.list',
                                   band_names=band_names,
                                   band_err_names=band_err_names,
                                   prior_band=prior_band,
                                   mag_limits = dict(mag_u_lsst=27.79,
                                                mag_g_lsst=29.04,
                                                mag_r_lsst=29.06,
                                                mag_i_lsst=28.62,
                                                mag_z_lsst=27.98,
                                                mag_y_lsst=27.05),
                                   output=output)

# note: dered is not run here because we did not add MW extinction
test_data = convert_catalog_to_test_data(data_degraded_gold, DS, band_names, nside=nside, dered=False)

bpz_estimated = estimate_bpz.estimate(test_data)
# we can also obtain the point estimate, e.g the mode:
zmode = pd.DataFrame(data={"pz_point": bpz_estimated().ancil['zmode']})
# attach point estimate to data:
data_degraded_gold = pd.concat([data_degraded_gold, zmode], axis=1)

## Step 3: RAIL.summarization (?) / TXPipe binning
### Tomographic binning (dummy case):

Here we include a dummy tomographic binning case, where we simply bin using the photo-z available for lens or source.

In general case, we want to plug in here the result from tomo challenge/TXPipe; this will take the result from metadetection (usually with limited bands and different selection of objects); 

In any case, we assume here we take in a data object with a column indicating the tomographic bin it is in, but we do not specify how the binning is assigned.

In [ ]:
data_degraded_gold_tomo = assign_source_bins(data_degraded_gold)

### TXPipe:

Using methods given by [TXPipie/lens_selector.py](https://github.com/LSSTDESC/TXPipe/blob/ad3844769f097d4e86f8ae090b1e9fbd0e99c801/txpipe/lens_selector.py) and [TXPipe/source_selector](https://github.com/LSSTDESC/TXPipe/blob/master/txpipe/source_selector.py). 

Notice for TXPipe, catalogue is derived from metacal or metadetect (e.g. using `riz`), and the end result is an additional column indicating which object is in which tomographic bin.

Having looked at TXPipe, it seems that you can pass on photo-z for objects in the catalogue. If these are passed, then tomographic bins are split given the bin edges in pz, if not, random forest will be used with the limited bands available. 

MetaDetect will have different variants of the data based on which object is detected, and so tomographic bin is determined in each case.

```
# TXpipe related imports

# Stages to run
stages:
    - name: FlowCreator             # Simulate a spectroscopic population
    - name: GridSelection          # Simulate a spectroscopic sample
    - name: TXParqetToHDF          # Convert the spec sample format
    - name: PZPrepareEstimatorLens   # Prepare the p(z) estimator
      classname: Inform_BPZ_lite   
    - name: PZEstimatorLens        # Measure lens galaxy PDFs
      classname: BPZ_lite
      threads_per_process: 1  
    - name: TXMeanLensSelector     # select objects for lens bins from the PDFs
    - name: Inform_NZDirLens       # Prepare the DIR method inputs for the lens sample     
      classname: Inform_NZDir
    - name: PZRailSummarizeLens    # Run the DIR method on the lens sample to find n(z)
      classname: PZRailSummarize  
    - name: PZRailSummarizeSource  # Run the DIR method on the lens sample to find n(z)
      classname: PZRailSummarize
    - name: TXSourceSelectorMetadetect  # select and split objects into source bins
    - name: Inform_NZDirSource     # Prepare the DIR method inputs for the source sample
      classname: Inform_NZDir
    - name: TXShearCalibration     # Calibrate and split the source sample tomographically
    - name: TXLensCatalogSplitter  # Split the lens sample tomographically
```

## Step 4: Evaluation

Here we write a simple code to evaluate the shifts and scatter of the photo-z bins for different depth:

In [1]:
# now for each quantile, find the data and sample the redshifts distirbution:

# split the systematic maps into 20 quantiles
nquantiles=20
# define the zgrid for nz
zgrid = np.linspace(0,3,101)
# number of bootstrap samples to use
nbootstrap=1000
# which redshift in the data set to use for nz
z_col = 'redshift' # true redshift, but can change here to pz_mode or something
# number of tomographic bins used 
npzbins=data_degraded_gold_tomo["tomo"].max()

# load the specific systematic map & mask to check correlation
mapin = hp.read_map(sys_to_check)
mask = hp.read_map(maskdir)
# quantile contains pixel indices, and meanv is the mean value of the systematic maps in each quantile
quantile, meanv = split_sys_map_quantiles(mapin, mask, nquantiles=nquantiles)

# compute simple summary statistic
nzstat_summary_split={}

for jj in range(npzbins):
    nzstat_summary_split["tomo-%d"%(jj+1)]={}
    
    ind = data_degraded_gold_tomo["tomo"] == (jj+1)

    for ii in range(nquantiles):
        ind *= np.in1d(data_degraded_gold_tomo["pixels"], quantile[ii])
        usecat = data_degraded_gold_tomo.loc[ind, :]
        # now for each tomographic bin, return redshift distribution:
        nzstat_summary_split["tomo-%d"%(jj+1)][ii] = compute_nzstats(usecat, z_col, 
                                                                     zgrid=zgrid, nbootstrap=nbootstrap)

    # compute the tot nz, meanz, sigmaz:
    nzstat_summary_tot["tomo-%d"%(jj+1)] = compute_nzstats(data_degraded_gold_tomo, z_col, 
                                                           zgrid=zgrid, nbootstrap=nbootstrap)

In [ ]:
# write to file:
outroot = savedir + "test-pz-with-i-band-coadd-Y1.yml"
write_evaluation_results(outroot, meanv, nzstat_summary_split, nzstat_summary_tot)

In [ ]:
# Show results in a plot:
fig,axarr=plt.subplots(3,npzbins,figsize=[15,10],gridspec_kw={'height_ratios': [3, 1, 1]})

## Top row: n(z) for each tomographic bin for each depth group
## Middle row: change in meanz as a function of depth with bootstrap errors
## Bottom row: change in sigmaz as a function of depth with boostrap errors

for ii in range(npzbins):
    
    # top row
    plt.sca(axarr[0,ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        nz = stat_summary_split["tomo-%d"%(ii+1)][q][0]
        plt.plot(nz[:,0], nz[:,1]/np.sum(nz[:,1])/(nz[1,0]-nz[0,0]), 
                color=cmap(colorlab))
    plt.text(0.6, 3.5, "tomo-%d"%(ii+1))
    plt.yticks([])
    plt.xlabel("$z$")
    
    # middle row
    plt.sca(axarr[1, ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        
        meanz = stat_summary_split["tomo-%d"%(ii+1)][q][1]
        meanztot = stat_summary_tot["tomo-%d"%(ii+1)][q][1]
        
        plt.errorbar(meanv[q], meanz[0], yerr=meanz[1],fmt='o',
                    color=cmap(colorlab))
    #dz = 0.005*(1+meanztot[0])
    plt.plot(meanv, np.ones(len(meanv))*meanztot[0], 'k-', alpha=0.5)
    #plt.fill_between([meanv[0], meanv[-1]], [-dz, -dz], 
                    #[dz, dz],color='k',alpha=0.2)
    if ii==0:
        plt.ylabel("$\\langle z\\rangle$")
    if ii>0:
        plt.yticks([])
    #plt.xlabel(sys)
    #plt.ylim([-0.015,0.015])
    #plt.xlim([24.6, 25.7])
    
    # bottom row
    plt.sca(axarr[2, ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        
        sigmaz = stat_summary_split["tomo-%d"%(ii+1)][q][2]
        sigmaztot = stat_summary_tot["tomo-%d"%(ii+1)][q][2]
        
        plt.errorbar(meanv[q], sigmaz[0], yerr=sigmaz[1],fmt='o',
                    color=cmap(colorlab))
    #dz = 0.005
    plt.plot(meanv, np.ones(len(meanv))*sigmaztot[0], 'k-', alpha=0.5)
    #plt.fill_between([meanv[0], meanv[-1]], [-dz, -dz], 
                    #[dz, dz],color='k',alpha=0.2)
    if ii==0:
        plt.ylabel("$\\sigma_z$")
    if ii>0:
        plt.yticks([])
    plt.xlabel(sys)
    #plt.ylim([-0.015,0.015])

plt.tight_layout()
plt.saveifg(savedir + 'fig.png', bbox_inches='tight')